<a id='s7'></a>
# Tree Cover gain widget and Ranked List

1. Widgets made from a dynamic sentence
2. User Variables (admin0 to admin1 and location)

** LOCATION Area Selectors **

    - All region
    - Biodiversity Hotspots
    - Protected Areas
    - Plantations
    - Managed Forests
    - Primary Forests
    - Intact Forests (2000/2013)

** Notes **

- AOI (area of interest) is the gadm region (admin0-2)
- location is the subset of that (i.e protected areas)
- Threshold is not a variable in this widget (it should be hardcoded in the queries and must be set to `thresh = 0`)

In [1]:
#Import Global Metadata etc

%run '0.Importable_Globals.ipynb'

In [15]:
# Variables
adm0 = 'BRA'
adm1 = 1
adm2 = None
extent_year = 2000     # extent data 2000 or 2010
location = "All Region"
thresh=75

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"


tags = ["forest_change", "land_cover"]
selectable_polynames = ['gadm28', 'wdpa', 'primary_forest', 'ifl_2013', 'plantations']

In [16]:
# get admin 1 or 2 level human-readable name info as needed:
areaId_to_name = None
if adm2:
    tmp = get_admin2_json(iso=adm0, adm1=adm1)
    areaId_to_name ={}
    for row in tmp:
        areaId_to_name[row.get('adm2')] = row.get('name')
if adm1 and not adm2:
    tmp = get_admin1_json(iso=adm0)
    areaId_to_name={}
    for row in tmp:
        areaId_to_name[row.get('adm1')] = row.get('name') 

In [17]:
#GAIN QUERY

def gain_query(p_name, adm0, adm1=None, adm2 = None):
    """Queries for the gain tables. Note, threshold must be hardcoded to 0
        and that will not change.
    """
    if adm2:
        print('Request for  adm2 area')
        sql = (f"SELECT area_gain as value FROM data WHERE iso = '{adm0}' "
               f"AND polyname = '{p_name}' "
               f"AND adm1 = {adm1} AND adm2 = {adm2} AND thresh = 0") 

    elif adm1:
        print('Request for  adm1 area')
        sql = (f"SELECT SUM(area_gain) as value FROM data WHERE iso = '{adm0}' "
               f"AND polyname = '{p_name}' "
               f"AND adm1 = {adm1} AND thresh = 0")
    elif adm0:
        print('Request for adm0 area')
        sql = (f"SELECT SUM(area_gain) as value FROM data WHERE iso = '{adm0}' "
               f"AND polyname = '{p_name}' "
               f"AND thresh = 0")
    return sql


In [18]:
def extent_queries(p_name, year, adm0, adm1=None, adm2 = None, threshold=30):
    if adm2:
        print('Request for adm2 area')
        sql = (f"SELECT SUM({year}) as value, "
               f"SUM(area_gadm28) as total_area "
               f"FROM data "
               f"WHERE iso = '{adm0}' "
               f"AND adm1 = {adm1} "
               f"AND adm2 = {adm2} "
               "AND thresh = {threshold} "
               f"AND polyname = '{p_name}'")
        return sql
    elif adm1:
        print('Request for  adm1 area')
        sql = (f"SELECT SUM({year}) as value, "
               f"SUM(area_gadm28) as total_area "
               f"FROM data "
               f"WHERE iso = '{adm0}' "
               f"AND adm1 = {adm1} "
               f"AND thresh = {threshold} "
               f"AND polyname = '{p_name}'")
        return sql
    elif adm0:
        print('Request for  adm0 area')
        sql = (f"SELECT SUM({year}) as value, "
               f"SUM(area_gadm28) as total_area "
               f"FROM data "
               f"WHERE iso = '{adm0}' "
               f"AND thresh = {threshold} "
               f"AND polyname = '{p_name}'")
        return sql

In [21]:
# gain query
sql = gain_query(p_name=polynames[location], adm0=adm0, adm1=adm1, adm2=adm2)
r1 = requests.get(url, params = {"sql": sql})

try:
    tree_cover_gain_2010 = r1.json().get('data')[0].get('value')
except:
    tree_cover_gain_2010 = 0.0
    
print(r1.url)    
pprint(r1.json())


Request for  adm1 area
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+SUM%28area_gain%29+as+value+FROM+data+WHERE+iso+%3D+%27BRA%27+AND+polyname+%3D+%27gadm28%27+AND+adm1+%3D+1+AND+thresh+%3D+0
{'data': [{'value': 38508.07257080078}],
 'meta': {'cloneUrl': {'body': {'dataset': {'application': ['your', 'apps'],
                                            'datasetUrl': '/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT%20SUM%28area_gain%29%20as%20value%20FROM%20data%20WHERE%20iso%20%3D%20%27BRA%27%20AND%20polyname%20%3D%20%27gadm28%27%20AND%20adm1%20%3D%201%20AND%20thresh%20%3D%200'}},
                       'http_method': 'POST',
                       'url': '/v1/dataset/499682b1-3174-493f-ba1a-368b4636708e/clone'}}}


In [22]:
# Total Extent query
sql = extent_queries(p_name=polynames[location], year=extent_year_dict[extent_year],
                     adm0=adm0, adm1=adm1, adm2=adm2, threshold=thresh)
r2 = requests.get(url, params = {"sql": sql})

try:
    extent_area = r2.json().get('data')[0].get('value')
except:
    extent_area = 0.0

print(r2.url)
pprint(r2.json())

Request for  adm1 area
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+SUM%28area_extent_2000%29+as+value%2C+SUM%28area_gadm28%29+as+total_area+FROM+data+WHERE+iso+%3D+%27BRA%27+AND+adm1+%3D+1+AND+thresh+%3D+75+AND+polyname+%3D+%27gadm28%27
{'data': [{'total_area': 15273286.984375, 'value': 14165459.6640625}],
 'meta': {'cloneUrl': {'body': {'dataset': {'application': ['your', 'apps'],
                                            'datasetUrl': '/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT%20SUM%28area_extent_2000%29%20as%20value%2C%20SUM%28area_gadm28%29%20as%20total_area%20FROM%20data%20WHERE%20iso%20%3D%20%27BRA%27%20AND%20adm1%20%3D%201%20AND%20thresh%20%3D%2075%20AND%20polyname%20%3D%20%27gadm28%27'}},
                       'http_method': 'POST',
                       'url': '/v1/dataset/499682b1-3174-493f-ba1a-368b4636708e/clone'}}}


In [23]:
area_percent = (100 * tree_cover_gain_2010 / extent_area)

print(area_percent)

0.27184484996625297


In [20]:
#Dynamic sentence

# we may need to modify the way we describe the location to fit gramatically
if location == 'All Region':
    region_phrase = 'region-wide'
else:
    region_phrase = 'in ' + location.lower()


print(f"From 2001 to 2012, ", end="")

if(not adm1 and not adm2):
    print(f"{iso_to_countries[adm0]} gained {extent_area:,.0f} ha of tree cover ", end="")
    print(f"{region_phrase}, equivalent to a {area_percent}% ",end="")
    print(f"increase relative to {extent_year} tree cover extent.", end="")
elif(not adm2):
    print(f"{areaId_to_name[adm1]} gained {extent_area:,.0f} ha of tree cover ", end="")
    print(f"{region_phrase}, equivalent to a {area_percent}% ",end="")
    print(f"increase relative to {extent_year} tree cover extent.", end="")
else:
    print(f"{areaId_to_name[adm2]} gained {extent_area:,.0f} ha of tree cover ", end="")
    print(f"{region_phrase}, equivalent to a {area_percent}% ", end="")
    print(f"increase relative to {extent_year} tree cover extent.", end="")


From 2001 to 2012, Acre gained 14,165,460 ha of tree cover region-wide, equivalent to a 0.27184484996625297% increase relative to 2000 tree cover extent.

### Ranked List

Shows a ranked list of regions ranked by either:

- Gain (ha)
- Gain relative to extent (%)

The list will show the selected region and regions with similar ranks

In [8]:
#Gathers all data required to rank regions. NOTE: adm0 querys are slow.

def ranking_gain_query(p_name, year, adm0, adm1=None, adm2 = None):
    """Queries for the gain tables. Note, threshold must be hardcoded to 0
        and that will not change.
    """
    if adm2:
        print('Request for  adm2 area')
        sql = (f"SELECT adm2 as region, area_gain AS value, {year} AS extent FROM data "
               f"WHERE iso = '{adm0}' "
               f"AND adm1 = {adm1} "
               f"AND polyname = '{p_name}' "
               f"AND thresh = 0") 

    elif adm1:
        print('Request for  adm1 area')
        sql = (f"SELECT adm1 as region, area_gain AS value, {year} AS extent FROM data "
               f"WHERE iso = '{adm0}' "
               f"AND polyname = '{p_name}' "
               f"AND thresh = 0")
    elif adm0:
        print('Request for adm0 area')
        sql = (f"SELECT iso as region, area_gain AS value, {year} AS extent FROM data "
               f"WHERE polyname = '{p_name}' "
               f"AND thresh = 0")
    return sql


In [14]:
#Sql query

sql = ranking_gain_query(polynames[location], extent_year_dict[extent_year], adm0, adm1, adm2)
    
r = requests.get(url, params = {"sql": sql})

print(r.url)

try:
    data = r.json().get('data')
    
except:
    print('No data')
    
data[0:3]


Request for  adm1 area
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+adm1+as+region%2C+area_gain+AS+value%2C+area_extent_2000+AS+extent+FROM+data+WHERE+iso+%3D+%27BRA%27+AND+polyname+%3D+%27gadm28%27+AND+thresh+%3D+0


[{'_id': 'AWB6geoRhRfkUZCTh-ul',
  'extent': 16212.9844124,
  'region': 8,
  'value': 1092.45395312},
 {'_id': 'AWB6geoRhRfkUZCTh-uo',
  'extent': 24828.917745,
  'region': 8,
  'value': 1047.54077483},
 {'_id': 'AWB6geoRhRfkUZCTh-uv',
  'extent': 26763.346828499998,
  'region': 8,
  'value': 2398.19586334}]

In [10]:
unique_regions = []
for d in data:
    if d.get('region') not in unique_regions:
        unique_regions.append(d.get('region'))

unique_regions[0:5]

[8, 6, 7, 9, 5]

In [11]:
gain_list = []
for u in unique_regions:
    tmp_gain = 0
    tmp_extent = 0
    for d in data:
        if d.get('region') == u and d.get('extent') != -9999:
            tmp_gain += d.get('value')
            tmp_extent += d.get('extent')
    if adm1 or adm2:
        u = areaId_to_name[u]
    try:
        gain_list.append({'region': u, 'gain': tmp_gain, 'relative': 100*tmp_gain/tmp_extent})
    except:
        gain_list.append({'region': u, 'gain': tmp_gain, 'relative': 0.0})
    
        
gain_list[0:5]    

[{'gain': 214351.5398714905,
  'region': 'Espírito Santo',
  'relative': 9.71427399910375},
 {'gain': 40006.765076539086,
  'region': 'Ceará',
  'relative': 0.8366734933929185},
 {'gain': 1810.81241895,
  'region': 'Distrito Federal',
  'relative': 1.3083005962565537},
 {'gain': 122692.03157379599,
  'region': 'Goiás',
  'relative': 1.0548395054644655},
 {'gain': 450082.7880964048,
  'region': 'Bahia',
  'relative': 1.8066284874859857}]

In [12]:
sorted_rel = sorted(gain_list, key=lambda k: k['relative'], reverse=True)
sorted_rel[0:5]

if adm2:
    region = areaId_to_name[adm2]
elif adm1:
    region = areaId_to_name[adm1]
elif adm0:
    region = adm0

rank_rel = 1
for s in sorted_rel:
    if s.get('region') == region:
        break
    else:
        rank_rel += 1      

if rank_rel == 1:
    bottom_bound = -1
    upper_bound = 4
elif rank_rel == 2:
    bottom_bound = -2
    upper_bound = 3    
elif rank_rel == len(sorted_rel):
    bottom_bound = -5
    upper_bound = 0   
elif rank_rel == len(sorted_rel)-1:
    bottom_bound = -4
    upper_bound = 1 
else:
    bottom_bound = -3
    upper_bound = 2 
    
rank_rel, sorted_rel[rank_rel+bottom_bound:rank_rel+upper_bound]        


(26,
 [{'gain': 55393.72579396401,
   'region': 'Amapá',
   'relative': 0.44735738758530763},
  {'gain': 58602.96722607883,
   'region': 'Piauí',
   'relative': 0.3872306404162091},
  {'gain': 50338.613219899,
   'region': 'Roraima',
   'relative': 0.2761163154574337},
  {'gain': 38508.07244910899,
   'region': 'Acre',
   'relative': 0.26552238762403985},
  {'gain': 176664.82799579806,
   'region': 'Amazonas',
   'relative': 0.11689619537290755}])

In [13]:
sorted_gain = sorted(gain_list, key=lambda k: k['gain'], reverse=True)
sorted_gain[0:5]

if adm2:
    region = areaId_to_name[adm2]
elif adm1:
    region = areaId_to_name[adm1]
elif adm0:
    region = adm0

rank_gain = 1
for s in sorted_gain:
    if s.get('region') == region:
        break
    else:
        rank_gain += 1
        
if rank_gain == 1:
    bottom_bound = -1
    upper_bound = 4
elif rank_gain == 2:
    bottom_bound = -2
    upper_bound = 3    
elif rank_gain == len(sorted_gain):
    bottom_bound = -5
    upper_bound = 0   
elif rank_gain == len(sorted_gain)-1:
    bottom_bound = -4
    upper_bound = 1 
else:
    bottom_bound = -3
    upper_bound = 2 
    
rank_gain, sorted_gain[rank_gain+bottom_bound:rank_gain+upper_bound]      

(20,
 [{'gain': 50338.613219899,
   'region': 'Roraima',
   'relative': 0.2761163154574337},
  {'gain': 40006.765076539086,
   'region': 'Ceará',
   'relative': 0.8366734933929185},
  {'gain': 38508.07244910899,
   'region': 'Acre',
   'relative': 0.26552238762403985},
  {'gain': 18906.55323916999,
   'region': 'Sergipe',
   'relative': 2.6788630765524477},
  {'gain': 18290.27636394394,
   'region': 'Pernambuco',
   'relative': 0.7314321709382418}])